In [ ]:
# Qua facciamo un algoritmo che prende i prezzi delle case da diverse fonti e li mette in un Excel con tutte le info del
# Caso. Speriamo che Immobiliare non sia in JavaScript.

# La ricerca sarà divisa in due: cercare la lista delle case, secondo dei criteri settabili dall'utente, e poi estrarre
# dalla lista un codice identificativo, o qualcosa che la contraddistinga, e poi da li entrare sul link e andare a vedere
# tutti i dettagli che ci sono

def getCodeAndDescription (pages = 10, max_price = 1500, locali_min = 3, air_conditioning = 'Si', arredato = 'Si', monolocali = 'Si'):

    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    import numpy as np
    from IPython.display import clear_output

    # Settiamo gli Headers

    headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

    # Mettiamo su l'URL di base: Immobiliare.it

    resultsPerPage = list()
    for page in range(1, pages):
    
        page = str(page)
        max_price = str(max_price)
        locali_min = str(locali_min)
        
        # mettiamo le variabili binarie:
        
        if air_conditioning == 'Si':
            ac = "con-aria-condizionata/"
        if air_conditioning == 'No':
            ac = ''
            
        if arredato == 'Si':
            fur = "&arredato=on"
        if arredato == 'No':
            fur = ''
            
        if monolocali == 'No':
            target_url = "https://www.immobiliare.it/affitto-case/milano/" + ac + "?pag=" + page + "&prezzoMassimo=" + max_price + "&localiMinimo=" + locali_min + fur
        if monolocali == 'Si':
            target_url = "https://www.immobiliare.it/affitto-monolocali/milano/" + ac + "?pag=" + page + "&prezzoMassimo=" + max_price + "&localiMinimo=" + locali_min + fur
            
        # settiamo la ricerca con BeautifulSoup

        resp = requests.get(target_url, headers=headers)
        soup = BeautifulSoup(resp.text, 'html.parser')

        a_tag = soup.findAll('div')

        fs = list()
        for i in a_tag:
            fs.append(i)
    
        a = pd.Series(fs).astype(str)

        # -----CODICE DELL'ANNUNCIO-----

        codice = a[a.str.contains('p class="in-realEstateListCard__description">')].reset_index()
        del[codice['index']]

        prova = codice[codice[0].str.contains('href="https://www.immobiliare.it/annunci/')].reset_index()
        del[prova['index']]

        code = list()
        for line in range(len(prova[0])):
            start = prova[0][line].find('href="https://www.immobiliare.it/annunci/') + len('href="https://www.immobiliare.it/annunci/')
    
            end = start + prova[0][line][start:].find('/')
    
            code.append(prova[0][line][start:end])

        code = pd.DataFrame(code).drop_duplicates().reset_index()
        del[code['index']]
        
        # -----DESCRIZIONE ANNUNCIO-----
        
        des = codice[codice[0].str.contains('p class="in-realEstateListCard__description">')].reset_index()
        del[des['index']]
    
        desc = list()
        for line in range(len(des[0])):
            start = des[0][line].find('p class="in-realEstateListCard__description">') + len('p class="in-realEstateListCard__description">')
    
            end = start + des[0][line][start:].find('</p></div><div class="nd-mediaObject__contentBottom in-card')
        
            if len(des[0][line][start:end]) != 0:
                desc.append(des[0][line][start:end])

        desc = pd.DataFrame(desc).drop_duplicates().reset_index()
        del[desc['index']]
        
        if (code.empty == False) | (desc.empty == False):
        
            finalDoc = pd.concat([code, desc], axis = 1).set_axis(['Ad_Code', 'Short description'], axis = 1)
            
            # Metti tutto insieme, e inserisci le colonne con le specifiche che inserisci nella funzione
            
            if air_conditioning == 'Si':
                AC_col = pd.DataFrame(np.full(len(finalDoc['Ad_Code']), 'Si')).set_axis(['aria_condizionata'], axis = 1)
                finalDoc = pd.concat([finalDoc, AC_col], axis = 1)
            
            if arredato == 'Si':
                fur_col = pd.DataFrame(np.full(len(finalDoc['Ad_Code']), 'Si')).set_axis(['Arredato'], axis = 1)
                finalDoc = pd.concat([finalDoc, fur_col], axis = 1)
                
            # Aggiungi una colonna con il link
            
            link = pd.DataFrame("https://www.immobiliare.it/annunci/" + finalDoc['Ad_Code'] + "/").set_axis(['Link'], axis = 1)
            
            finalDoc = pd.concat([finalDoc, link], axis = 1)
            
            resultsPerPage.append(finalDoc)
    
    resultsPerPage = pd.concat([series for series in resultsPerPage], axis = 0).reset_index()
    del[resultsPerPage['index']]

    #print(resultsPerPage)
    
    return resultsPerPage.dropna()

# Seconda parte: andiamo a iterare PER OGNI CODICE che abbiamo trovato prima, ed estraiamo prezzo, spese, caratteristiche
# della casa.

def HouseDetails (pages = 10, max_price = 1500, locali_min = 3, air_conditioning = 'Si', arredato = 'Si', monolocali = 'Si'):

    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    import numpy as np
    from IPython.display import clear_output

    test = getCodeAndDescription (pages = pages, max_price = max_price, locali_min = locali_min,
                              air_conditioning = air_conditioning, arredato = arredato, monolocali = monolocali)

    test = test.drop_duplicates(subset = ['Link'])

    total = list()
    for iterat, link in enumerate(test['Link'].unique()):
    
        #print(link)

        # Il target dello scrape diventa questo
    
        headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
        target_url = link
    
        resp = requests.get(target_url, headers=headers)
        soup = BeautifulSoup(resp.text, 'html.parser')  

        a_tag = soup.findAll('div')

        fs = list()
        for i in a_tag:
            fs.append(i)
    
        a = pd.Series(fs).astype(str)
    
        
        # -----NUMERO DI FOTO DISPONIBILI (indice di scam)-----
    
        if a[a.str.contains('<span class="nd-tag in-mosaic__tag" role="button" tabindex="0"><span class="nd-tag__text">Vedi ')].reset_index()[0].empty == False:
    
            fotoBase = a[a.str.contains('<span class="nd-tag in-mosaic__tag" role="button" tabindex="0"><span class="nd-tag__text">Vedi ')].reset_index()[0][0]
            fotoStart = fotoBase.find('<span class="nd-tag in-mosaic__tag" role="button" tabindex="0"><span class="nd-tag__text">Vedi ') + len('<span class="nd-tag in-mosaic__tag" role="button" tabindex="0"><span class="nd-tag__text">Vedi ')
            fotoEnd = fotoBase[fotoStart:].find(' foto</span></span>')
    
            numeroFoto = fotoBase[fotoStart : fotoStart + fotoEnd]        
        else:
            numeroFoto = 'meno di 3'
    
        # -----LOCATION - QUARTIERE-----
        
        if a[a.str.contains('<a class="in-titleBlock__link" href="#mappa"><span class="in-location">Milano</span><span class="in-location">')].reset_index()[0].empty == False:

            locBase = a[a.str.contains('<a class="in-titleBlock__link" href="#mappa"><span class="in-location">Milano</span><span class="in-location">')].reset_index()[0][0]
            locStart = locBase.find('<a class="in-titleBlock__link" href="#mappa"><span class="in-location">Milano</span><span class="in-location">') + len('<a class="in-titleBlock__link" href="#mappa"><span class="in-location">Milano</span><span class="in-location">')
            locEnd = locBase[locStart:].find('</span><span')
    
            location_Q = locBase[locStart : locStart + locEnd]
    
        # -----LOCATION - VIA-----
        
            baseV = locBase[locStart:]

            locStartV = baseV.find('</span><span class="in-location">') + len('</span><span class="in-location">')
            locEndV = baseV[locStartV:].find('</span></a></div><')
    
            location_V = baseV[locStartV : locStartV + locEndV]
        
        else:
            location_Q = 'No location Found'
            location_V = 'No location Found'

    
        #a.to_excel(r"C:\Users\39328\OneDrive\Desktop\zio pino indice.xlsx")
        
        # -----DESCRIZIONE E NOME (servono anche a capire se ci sono delle spese incluse nel prezzo)-----
    
        # Filtra per la riga che contiene la descrizione
    
        if a[a.str.contains('id="descrizione"></span>Descrizione</div><p class="in-description__title">')].reset_index()[0].empty == False:
        
            desRaw = a[a.str.contains('id="descrizione"></span>Descrizione</div><p class="in-description__title">')].reset_index()[0][0]
        
            startRaw = desRaw.find('id="descrizione"></span>Descrizione</div><p class="in-description__title">') + len('id="descrizione"></span>Descrizione</div><p class="in-description__title">')
    
            # Isolo il nome dell'opportunità
    
            nameStart = startRaw
            nameEnd = desRaw[startRaw:].find('</p><div class="in-readAll in-readAll--lessContent"><div>')
    
            name = desRaw[nameStart: nameStart + nameEnd]
        
            # Isolo la descrizione "lunga" 
        
            startRawD = desRaw.find('</p><div class="in-readAll in-readAll--lessContent"><div>') + len('</p><div class="in-readAll in-readAll--lessContent"><div>')
        
            desStart = startRawD
            desEnd = desRaw[startRawD:].find('</div></div><button class="nd-button nd-button--link nd-button--centered in-readAll__action"><svg class="')
        
            description = desRaw[desStart: desStart + desEnd]
        
            #print(name)
    #print(description)

        else:
            name = 'Nome mancante'
            description = 'Descrizione mancante'
        
        
        # -----CARATTERISTICHE DELLA CASA-----
    
        # Ripartiamo da a e tiriamo fuori le informazioni che servono
        # pre-filtriamo a
    
        char = a[a.str.contains('id="caratteristiche"></span>Caratteristiche</div><dl class="in-realEstateFeatures__list"><dt class="in-realEstateFeatures__title"')].reset_index()[0][0]
    
        # ---RIFERIMENTO E DATA DI ANNUNCIO---
    
        startRif = char.find('id="caratteristiche"></span>Caratteristiche</div><dl class="in-realEstateFeatures__list"><dt class="in-realEstateFeatures__title">Riferimento e Data annuncio</dt><dd class="in-realEstateFeatures__value">') + len('id="caratteristiche"></span>Caratteristiche</div><dl class="in-realEstateFeatures__list"><dt class="in-realEstateFeatures__title">Riferimento e Data annuncio</dt><dd class="in-realEstateFeatures__value"')
        endRif = char[startRif:].find('</dd><dt')
    
        date = char[startRif:startRif + endRif]
        date = date[len(date) - 10 : len(date)]
    
        # ---CONTRATTO---

        charCon = char[startRif:]
    
        startCon = charCon.find('contratto</dt><dd class="in-realEstateFeatures__value">') + len('contratto</dt><dd class="in-realEstateFeatures__value">')
        endCon = charCon[startCon:].find('</dd><dt')
    
        contract = charCon[startCon : startCon + endCon]
    
        # ----SUPERFICIE----
    
        supStart = char.find('</dd><dt class="in-realEstateFeatures__title">superficie</dt><dd class="in-realEstateFeatures__value">') + len('</dd><dt class="in-realEstateFeatures__title">superficie</dt><dd class="in-realEstateFeatures__value">')
        supEnd = char[supStart:].find(' m²<!-- -->')
    
        surface = char[supStart : supStart + supEnd]
    
        # ----COMPOSIZIONE DELLA CASA----

        compStart = char.find('<a role="button" tabindex="0">Vedi dettaglio</a></dd><dt class="in-realEstateFeatures__title">locali</dt><dd class="in-realEstateFeatures__value">') + len('<a role="button" tabindex="0">Vedi dettaglio</a></dd><dt class="in-realEstateFeatures__title">locali</dt><dd class="in-realEstateFeatures__value">')
        compEnd = char[compStart:].find('</dd><dt')
    
        composition = char[compStart : compStart + compEnd]
    
        # ----PIANO----
    
        floorStart = char.find('class="in-realEstateFeatures__title">piano</dt><dd class="in-realEstateFeatures__value">') + len('class="in-realEstateFeatures__title">piano</dt><dd class="in-realEstateFeatures__value">')
        floorEnd = char[floorStart:].find('</dd><dt')
    
        floor = char[floorStart : floorStart + floorEnd]
    
        # ----TOTALE PIANI----

        TfloorStart = char.find('class="in-realEstateFeatures__title">totale piani edificio</dt><dd class="in-realEstateFeatures__value">') + len('class="in-realEstateFeatures__title">totale piani edificio</dt><dd class="in-realEstateFeatures__value">')
        TfloorEnd = char[TfloorStart:].find('</dd><dt')
    
        Tfloor = char[TfloorStart : TfloorStart + TfloorEnd]
    
        # ----ALTRE FEATURES---- 
    
        featuresStart = char.find('class="in-realEstateFeatures__title">altre caratteristiche</dt><dd class="in-realEstateFeatures__value in-realEstateFeatures__tagContainer"><span class="in-realEstateFeatures__tag nd-tag">') + len('class="in-realEstateFeatures__title">altre caratteristiche</dt><dd class="in-realEstateFeatures__value in-realEstateFeatures__tagContainer"><span class="in-realEstateFeatures__tag nd-tag">')
        featuresEnd = char[featuresStart:].find('</span></dd></dl><div')
    
        features = char[featuresStart : featuresStart + featuresEnd]
        features = features.replace ('</span><span class="in-realEstateFeatures__tag nd-tag">', ', ')
    
        # ----PREZZO----

        priceStart = char.find('class="in-realEstateFeatures__list"><dt class="in-realEstateFeatures__title">prezzo</dt><dd class="in-realEstateFeatures__value">€ ') + len('class="in-realEstateFeatures__list"><dt class="in-realEstateFeatures__title">prezzo</dt><dd class="in-realEstateFeatures__value">€ ')
        priceEnd = char[priceStart:].find('/mese</dd><dt')
    
        price = char[priceStart : priceStart + priceEnd]
    
        # ----SPESE DI CONDOMINIO----
    
        CpriceStart = char.find('class="in-realEstateFeatures__title">spese condominio</dt><dd class="in-realEstateFeatures__value">') + len('class="in-realEstateFeatures__title">spese condominio</dt><dd class="in-realEstateFeatures__value">')
        CpriceEnd = char[CpriceStart:].find('</dd><dt')
    
        Cprice = char[CpriceStart : CpriceStart + CpriceEnd]
        Cprice = Cprice.replace ('€ ', '')
        Cprice = Cprice.replace ('.', '')
        Cprice = Cprice.replace ('/mese', '')

        # ----CAUZIONE----
    
        CauzStart = char.find('class="in-realEstateFeatures__title">cauzione</dt><dd class="in-realEstateFeatures__value">') + len('class="in-realEstateFeatures__title">cauzione</dt><dd class="in-realEstateFeatures__value">')
        CauzEnd = char[CauzStart:].find('</dd></dl>')
    
        cauzione = char[CauzStart : CauzStart + CauzEnd]
        cauzione = cauzione.replace ('€ ', '')
        cauzione = cauzione.replace ('.', '')
    
        print("Scraping Immobiliare.it... Progress:", round((iterat/len(test['Link'].unique()))*100, 2), '%')
        
        # Estrai il codice dell'appartamento
        
        ex_code = (link[link.find('annunci/') + len('annunci/') : len(link) - 1])
        
        # Mettiamo tutto insieme
    
        houseDoc = pd.concat([pd.Series(date), pd.Series(ex_code), pd.Series(location_Q), pd.Series(location_V), pd.Series(name), pd.Series(description), pd.Series(contract), pd.Series(surface),
                         pd.Series(composition), pd.Series(floor), pd.Series(Tfloor), pd.Series(features), pd.Series(price), pd.Series(Cprice),
                         pd.Series(cauzione), pd.Series(numeroFoto), pd.Series(link)], axis = 1).set_axis(['Data', 'Codice Annuncio', 'Quartiere', 'Via', 'Nome', 'Descrizione', 'Tipo di contratto', 'Superficie',
                                                'Composizione', 'Piano', 'Totale Piani', 'Caratteristiche',
                                                'Prezzo', 'Spese di condominio', 'Cauzione', 'Numero di foto', 'Link al sito'], axis = 1)
    
        total.append(houseDoc)
    
        clear_output(wait = True)
    
    total = pd.concat([series for series in total], axis = 0).reset_index()
    del[total['index']]
    
    total.to_excel(r"C:\Users\39328\OneDrive\Desktop\Case_affitto.xlsx", index = False)

    return total

def houseNews():
    
    import pandas as pd
    import datetime
    
    today = HouseDetails (pages = 20, max_price = 900, locali_min = 1, air_conditioning = 'Si', arredato = 'Si', monolocali = 'Si')

    yesterday = pd.read_excel(r"C:\Users\39328\OneDrive\Desktop\Case_affitto.xlsx")

    today = HouseDetails (pages = 20, max_price = 900, locali_min = 1, air_conditioning = 'Si', arredato = 'Si', monolocali = 'Si')

    yesterday['Codice Annuncio'] = yesterday['Codice Annuncio'].astype(str)
    today['Codice Annuncio'] = today['Codice Annuncio'].astype(str)

    news = today[today['Codice Annuncio'].isin(yesterday['Codice Annuncio']) == False]

    print('Data:', datetime.datetime.today(), 'Nuovi Annunci:', len(news))
    
    print('\n')
    
    if news.empty == False:
        #for viaNA, qNA, linkNA, prezzoNA, speseNA in news['Via'], news['Quartiere'], news['Link al sito'], news['Prezzo'], news['Spese di condominio'] :
        #    print('Nuovo annuncio in:', viaNA, '(', qNA, ')', 'Prezzo:', prezzoNA, 'Spese:', speseNA, 'Link:', linkNA)
        for qNA in news['Quartiere']:
            print('Nuovo annuncio in:', qNA)
    
    return news
